In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [3]:
train_nlu_model_switch = True

In [4]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.3s finished


In [5]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [11]:
query = 'how to extend tourist visa'

In [12]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.9920640145229644,
   'end': 26,
   'entity': 'subject',
   'extractor': 'ner_crf',
   'start': 14,
   'value': 'tourist visa'}],
 'intent': {'confidence': 0.40654704476315828, 'name': 'conditions'},
 'intent_ranking': [{'confidence': 0.40654704476315828, 'name': 'conditions'},
  {'confidence': 0.22975071028326174, 'name': 'duration'},
  {'confidence': 0.20037264848074174, 'name': 'definition'},
  {'confidence': 0.062911654253973798, 'name': 'extension'},
  {'confidence': 0.017926943131788962, 'name': 'validity'},
  {'confidence': 0.016534318939068517, 'name': 'contact'},
  {'confidence': 0.016155625784386729, 'name': 'address'},
  {'confidence': 0.014373344205687281, 'name': 'benefits'},
  {'confidence': 0.013020717728883618, 'name': 'email'},
  {'confidence': 0.010860832120146355, 'name': 'goodbye'}],
 'text': 'how to extend tourist visa'}

In [13]:
import spacy
nlp = spacy.load('en')

In [16]:
a = nlp('foreign entry')
b= nlp('foreigner entry')
a.similarity(b)

0.80244522035707999